In [263]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils import clip_grad_norm_
from torch.autograd import Variable
from torch.autograd.function import Function
from torchsummary import summary
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import torch.optim as optim
from sklearn.preprocessing import StandardScaler

import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing.sequence import TimeseriesGenerator

In [264]:
hyperparam = {
    "data" : {
        "stake_training_data" : 0.75, 
        "path" : '../../../data/vega_shrinkwrapper_original/NewBlade/'
    },
    "model" : {
        "input_size" : 7,
        "n_hidden" : 25,
        "sequence_size" : 20,
        "batch_size" : 1,
        "lstm_layer" : 3,
    },
    "cycling_lr" : {
        # step_size is the number of training iterations (total samples/batch_size) per half cycle. 
        # Authors suggest setting step_size 2-8 x training iterations in epoch.
        "step_size" : (1536/8)*2, 
        # Mode can be one of {triangular, triangular2, exp_range}
        "mode" : "triangular", 
        "gamma" : 0.9995,
        "base_lr" : 3e-2, 
        "max_lr" :0.1
    },
    "training": {
        "n_epochs" : 20,
        "patience" : 50,
    }
}

# Data

In [265]:
torch.manual_seed(42)
x = torch.randn(8, 7, dtype=torch.float)
target_data = torch.randn(8, 7, dtype=torch.float)
print(x)
print(target_data)

tensor([[ 1.9269,  1.4873,  0.9007, -2.1055,  0.6784, -1.2345, -0.0431],
        [-1.6047, -0.7521,  1.6487, -0.3925, -1.4036, -0.7279, -0.5594],
        [-0.7688,  0.7624,  1.6423, -0.1596, -0.4974,  0.4396, -0.7581],
        [ 1.0783,  0.8008,  1.6806,  1.2791,  1.2964,  0.6105,  1.3347],
        [-0.2316,  0.0418, -0.2516,  0.8599, -1.3847, -0.8712, -0.2234],
        [ 1.7174,  0.3189, -0.4245,  0.3057, -0.7746, -0.8371, -0.9224],
        [ 1.8113,  0.1606,  0.3672,  0.1754,  1.3852, -0.4459, -1.2024],
        [ 0.7078, -1.0759,  0.5357,  1.1754,  0.5612, -0.4527, -0.7718]])
tensor([[ 0.1453,  0.2311,  0.0087, -0.1423,  0.1971, -1.1441,  0.3383],
        [ 1.6992,  2.8140,  0.3598, -0.0898,  0.4584, -0.5644,  1.0563],
        [-1.4692,  1.4332,  0.7281, -0.7106, -0.6021,  0.9604,  0.4048],
        [-1.3543, -0.4976,  0.4747, -0.1976,  1.2683,  1.2243,  0.0981],
        [ 1.7423, -1.3527,  0.2191,  0.5526, -0.6788,  0.5743,  0.1877],
        [-0.3576, -0.3165,  0.5886, -0.8905,  0.40

In [266]:
class SingleLayer(nn.Module):
    def __init__(self, batch_size, input_dim, n_hidden, n_layers):
        super(SingleLayer, self).__init__()
        # Attributes for LSTM Network
        self.input_dim = input_dim
        self.n_hidden = n_hidden
        self.n_layers = n_layers
        self.batch_size = batch_size
        
        # Attribut for Gaussians
        self.n_gaus_param = 1
        
        # Definition of NN layer
        self.fc1 = nn.Linear(self.input_dim, self.n_hidden)
        self.fc2 = nn.Linear(self.n_hidden, self.input_dim)
        
    def forward(self, x):
        x = torch.tanh(self.fc1(x))
        out = self.fc2(x)
        return out
    

In [267]:
class LossModule(torch.nn.Module):
    def __init__(self):
        """
        In the constructor we instantiate the module and assign them as
        member variables.
        """
        super(LossModule, self).__init__()

    def forward(self, output, target_data):
        """
        In the forward function we accept a Tensor of input data and we must return
        a Tensor of output data. 
        """
        # Sum over each feature
        mean_loss = torch.sum(((output - target_data) ** 2), dim=1) / 7
        # Sum over all samples and take the mean to get mini-batch MSE
        mean_loss = torch.sum(mean_loss) / 8
        return mean_loss

In [268]:
class OwnLossFunction(Function):
    @staticmethod
    # bias is an optional argument
    def forward(ctx, output, target_data):
        # Select and transform data
        ctx.save_for_backward(output, target_data)
        
        # Compute loss
        # Sum over each feature
        mean_loss = torch.sum(((output - target_data) ** 2), dim=1) / 7
        # Sum over all samples and take the mean to get mini-batch MSE
        mean_loss = torch.sum(mean_loss) / 8
        return mean_loss
    
    @staticmethod
    def backward(ctx, grad_input):
        """
        grad_output = 1, because it is initialize by Optimizer 
        In the backward pass we need to compute the gradient of the loss
        with respect to the input.
        """
        output, target_data = ctx.saved_tensors # this is a tensor of models output
        grad_output = 1/7*2.*(output-target_data)
        #print(grad_output)
        return grad_output, None

In [269]:
torch.manual_seed(0)
model = SingleLayer(batch_size=hyperparam['model']['batch_size'], input_dim=hyperparam['model']['input_size'], 
             n_hidden=hyperparam['model']['n_hidden'], n_layers=hyperparam['model']['lstm_layer'])
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)  

## Loss function with nn.MSELoss()

In [270]:
print("Start model training")
for epoch in range(hyperparam["training"]["n_epochs"]+1):
    # Empty list for recording performance 
    epoch_training_loss = []
    epoch_validation_loss = []
    
    model.train()
        
    # Zero out gradient, else they will accumulate between minibatches
    optimizer.zero_grad()

    # Forward propagation
    output = model(x)

    # Calculate loss
    criterion = nn.MSELoss()
    loss = criterion(output, target_data)

    # Backward pass
    loss.backward()
    #params = list(model.parameters())
    #print(params[0].grad)

    # Update parameters
    optimizer.step()

    print("-------- epoch_no. {} finished with training loss {}--------".format(epoch, loss.item()))

Start model training
-------- epoch_no. 0 finished with training loss 0.9291448593139648--------
-------- epoch_no. 1 finished with training loss 0.9243056774139404--------
-------- epoch_no. 2 finished with training loss 0.9195181727409363--------
-------- epoch_no. 3 finished with training loss 0.9147815108299255--------
-------- epoch_no. 4 finished with training loss 0.9100947380065918--------
-------- epoch_no. 5 finished with training loss 0.9054569602012634--------
-------- epoch_no. 6 finished with training loss 0.9008673429489136--------
-------- epoch_no. 7 finished with training loss 0.8963251113891602--------
-------- epoch_no. 8 finished with training loss 0.8918293118476868--------
-------- epoch_no. 9 finished with training loss 0.8873791694641113--------
-------- epoch_no. 10 finished with training loss 0.8829740285873413--------
-------- epoch_no. 11 finished with training loss 0.8786129355430603--------
-------- epoch_no. 12 finished with training loss 0.8742952346801

## Loss function with own Module

In [271]:
torch.manual_seed(0)
model = SingleLayer(batch_size=hyperparam['model']['batch_size'], input_dim=hyperparam['model']['input_size'], 
             n_hidden=hyperparam['model']['n_hidden'], n_layers=hyperparam['model']['lstm_layer'])
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)  

In [272]:
print("Start model training")
for epoch in range(hyperparam["training"]["n_epochs"]+1):
    # Empty list for recording performance 
    epoch_training_loss = []
    epoch_validation_loss = []
    
    model.train()
        
    # Zero out gradient, else they will accumulate between minibatches
    optimizer.zero_grad()

    # Forward propagation
    output = model(x)
    #print(output)

    # Calculate loss
    criterion = LossModule()
    loss = criterion(output, target_data)

    # Backward pass
    loss.backward()
    #params = list(model.parameters())
    #print(params[0].grad)

    # Update parameters
    optimizer.step()

    print("-------- epoch_no. {} finished with training loss {}--------".format(epoch, loss.item()))

Start model training
-------- epoch_no. 0 finished with training loss 0.9291448593139648--------
-------- epoch_no. 1 finished with training loss 0.9243056774139404--------
-------- epoch_no. 2 finished with training loss 0.9195181727409363--------
-------- epoch_no. 3 finished with training loss 0.9147815108299255--------
-------- epoch_no. 4 finished with training loss 0.910094678401947--------
-------- epoch_no. 5 finished with training loss 0.9054569602012634--------
-------- epoch_no. 6 finished with training loss 0.9008672833442688--------
-------- epoch_no. 7 finished with training loss 0.8963251113891602--------
-------- epoch_no. 8 finished with training loss 0.891829252243042--------
-------- epoch_no. 9 finished with training loss 0.8873792290687561--------
-------- epoch_no. 10 finished with training loss 0.8829740285873413--------
-------- epoch_no. 11 finished with training loss 0.8786129355430603--------
-------- epoch_no. 12 finished with training loss 0.874295294284820

## Loss function with own Function -- !! Ist nicht gleich wie die anderen beiden Varianten !!

In [273]:
torch.manual_seed(0)
model = SingleLayer(batch_size=hyperparam['model']['batch_size'], input_dim=hyperparam['model']['input_size'], 
             n_hidden=hyperparam['model']['n_hidden'], n_layers=hyperparam['model']['lstm_layer'])
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)  

In [274]:
print("Start model training")

for epoch in range(hyperparam["training"]["n_epochs"]+1):
    # Empty list for recording performance 
    epoch_training_loss = []
    epoch_validation_loss = []
    
    model.train()
        
    # Zero out gradient, else they will accumulate between minibatches
    optimizer.zero_grad()

    # Forward propagation
    output = model(x)

    # Calculate loss
    loss = OwnLossFunction.apply(output, target_data)

    # Backward pass
    loss.backward()
    #params = list(model.parameters())
    #print(params[0].grad)

    # Update parameters
    optimizer.step()

    print("-------- epoch_no. {} finished with training loss {}--------".format(epoch, loss.item()))

Start model training
-------- epoch_no. 0 finished with training loss 0.9291448593139648--------
-------- epoch_no. 1 finished with training loss 0.8911610245704651--------
-------- epoch_no. 2 finished with training loss 0.8562833666801453--------
-------- epoch_no. 3 finished with training loss 0.8240970373153687--------
-------- epoch_no. 4 finished with training loss 0.7942553758621216--------
-------- epoch_no. 5 finished with training loss 0.766467809677124--------
-------- epoch_no. 6 finished with training loss 0.7404906153678894--------
-------- epoch_no. 7 finished with training loss 0.7161188125610352--------
-------- epoch_no. 8 finished with training loss 0.6931804418563843--------
-------- epoch_no. 9 finished with training loss 0.6715306043624878--------
-------- epoch_no. 10 finished with training loss 0.6510477662086487--------
-------- epoch_no. 11 finished with training loss 0.6316291093826294--------
-------- epoch_no. 12 finished with training loss 0.61318814754486